# Stochastic programming and sampling

Furniture model from Royset/Wets: Primer Book chapter 2.

In [1]:
import gamspy as gp
import gamspy.math as gpm
from gamspy import Sum, Card

import sys
import numpy as np
import pandas as pd
from statistics import NormalDist

class arguments:
  def __init__(self, s=400, solver='cplex', seed = 0):
    self.s = s
    self.solver = solver
    self.seed = seed
      
args = arguments(s=400)

Calculate the mean value solution

In [2]:
m = gp.Container()

j = gp.Set(m,'j',description='dresser',records=[i+1 for i in range(4)])
i = gp.Set(m,'i',description='labor',records=[('c','carpentry'),('f','finishing')])

d = gp.Parameter(m,'d',domain=i,records=[('c',6000),('f',4000)],description='mean time avail')
c = gp.Parameter(m,'c',domain=j,records=np.array([-12,-25,-21,-40]),description='cost')
q = gp.Parameter(m,'q',domain=i,records=np.array([5,10]),description='overtime cost')

s = gp.Set(m,'s',description='planning scenarios',records = ['mvs'])
p = gp.Parameter(m,'p',domain=s,description='scenario probability',records=[('mvs',1)])
t = gp.Parameter(m,'t',domain=[s,i,j],records=np.array(
    [[[4, 9, 7, 10],[1, 1, 3, 40]]]),description='mean requirement')

x = gp.Variable(m,'x','positive',domain=j,description='production')
y = gp.Variable(m,'y','positive',domain=[s,i],description='overtime')

cons = gp.Equation(m,'cons',domain=[s,i])
cons[s,i] = Sum(j, t[s,i,j]*x[j]) <= d[i] + y[s,i]

furn= gp.Model(m,'furn',
    equations=m.getEquations(),
    problem=gp.Problem.LP,
    sense=gp.Sense.MIN,
    objective=Sum(j, c[j]*x[j]) + Sum([s,i], p[s]*q[i]*y[s,i])
)

furn.solve(output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,-18666.6666666667,3,7,LP,CPLEX,0


In [3]:
display(x.l.records)
xmvs = gp.Parameter(m,'xmvs',domain=j)
xmvs[j] = x.l[j]
fmvs = furn.objective_value

,j,level
0,1,1333.333333
1,2,0.000000
2,3,0.000000
3,4,66.666667


Generate data to run (small) extensive form problem

In [4]:
from itertools import product
from functools import reduce

def genScen(rv_t):
    # cross product of values in ct and d
    prodlist = [elem[1::2] for elem in rv_t.values()]
    tv = list(product(*prodlist))
    t = []
    s = ['s'+str(scen) for scen in range(1,sum(1 for _ in tv)+1)]
    keysct = list(ct.keys())
    scen = 0
    for i in tv:
        scen +=1
        for j in range(len(ct)):
            t.append(('s'+str(scen),keysct[j][0],keysct[j][1],i[j]))

    prodlist = [elem[::2] for elem in rv_t.values()]
    tv = list(product(*prodlist))
    p = [];
    scen = 0
    for i in tv:
        scen +=1
        p.append(('s'+str(scen),reduce((lambda x, y: x * y), i)))
    return s, p, t

# discrete distribution (p,val) for each element of T and d
ct = {('c','1'): [.25,  3.60,.25,  3.90,.25, 4.10,.25, 4.40],
    ('c','2'): [.25,  8.25,.25,  8.75,.25, 9.25,.25, 9.75],
    ('c','3'): [.25,  6.85,.25,  6.95,.25, 7.05,.25, 7.15],
    ('c','4'): [.25,  9.25,.25,  9.75,.25,10.25,.25,10.75],
    ('f','1'): [.25,  0.85,.25,  0.95,.25, 1.05,.25, 1.15],
    ('f','2'): [.25,  0.85,.25,  0.95,.25, 1.05,.25, 1.15],
    ('f','3'): [.25,  2.60,.25,  2.90,.25, 3.10,.25, 3.40],
    ('f','4'): [.25,37.00,.25,39.00,.25,41.00,.25,43.00]}
svals,pvals,tvals = genScen(ct)

s.setRecords(svals)
p.setRecords(pvals)
t.setRecords(tvals)

In [5]:
furn.solve(solver='cplex',solver_options={'threads': 2, 'lpmethod': 4, 'advind': 0},output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,-18127.8446140515,131073,131077,LP,CPLEX,0.365


In [6]:
display(x.l.records)

,j,level
0,1,205.162318
1,2,0.000000
2,3,696.011413
3,4,32.980018


In [7]:
# What is value of stochastic solution

In [8]:
fss = furn.objective_value
x.fx[j] = xmvs[j]
furn.solve(solver='mosek',solver_options={'threads': 2, 'lpmethod': 4, 'advind': 0},output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,-17000.0000000002,131073,131077,LP,MOSEK,0.039


In [9]:
print(f'Overtime cost = {furn.objective_value-fmvs}')
print(f'vss = {-fss+furn.objective_value}')

Overtime cost = 1666.666666666515
vss = 1127.844614051366


Now generate discrete sample of distribution when T(i,j) is Normal(mean,1/4).  Better code in sampling notebook.